## Energy Transition Case Study

This notebook presents the base case optimization of the German electricity market as presented in the manuscript presenting the PULPO framework.

Updated by Fabian Lechtenberg, 15.04.2024.

See [this notebook](./electricity_showcase.ipynb) for additional results and functionalities of this case study, and [this notebook](./plastic_showcase.ipynb) for a more thorough demonstration of the functionalities on a plastics recycling case study.

<div style="text-align: center; background-color: #f0f0f0; padding: 10px;">
    <h2 style="font-family: 'Arial', sans-serif; font-weight: bold; color: #555;">Demonstration 🚀</h2>
</div>

### Base Problem

In [1]:
# Import section
import os
import sys
sys.path.append('../')

import pandas as pd
from pulpo import pulpo

In [2]:
# Your brightway2 project, database and targeted impact category
project = "pulpo"
database = "cutoff38"
methods = {"('IPCC 2013', 'climate change', 'GWP 100a')": 1,
           "('ReCiPe Endpoint (E,A)', 'resources', 'total')": 0,
           "('ReCiPe Endpoint (E,A)', 'human health', 'total')": 0,
           "('ReCiPe Endpoint (E,A)', 'ecosystem quality', 'total')": 0,
           "('ReCiPe Midpoint (E) V1.13', 'ionising radiation', 'IRP_HE')": 0}

# Substitute with your working directory of choice
notebook_dir = os.path.dirname(os.getcwd())
directory = os.path.join(notebook_dir, 'data')

In [3]:
# Create a PULPO object
pulpo_worker = pulpo.PulpoOptimizer(project, database, methods, directory)

In [4]:
# Retrieve LCI data
pulpo_worker.get_lci_data()

In [5]:
# Retrieve the electricity market
activities = ["market for electricity, high voltage"]
reference_products = ["electricity, high voltage"]
locations = ["DE"]
electricity_market = pulpo_worker.retrieve_activities(activities=activities,
                                                      reference_products=reference_products,
                                                      locations=locations)

In [6]:
# Specify the functional unit as demand dictionary
demand = {electricity_market[0]: 1.28819e+11}

In [7]:
# Retrieve the choices
activities = ["electricity production, lignite", 
             "electricity production, hard coal",
             "electricity production, nuclear, pressure water reactor",
             "electricity production, wind, 1-3MW turbine, onshore"]
reference_products = ["electricity, high voltage"]
locations = ["DE"]

electricity_activities = pulpo_worker.retrieve_activities(activities=activities,
                                                          reference_products=reference_products,
                                                          locations=locations)

In [8]:
# Specify the choices dictionary
choices  = {'electricity': {electricity_activities[0]: 1e16,
                            electricity_activities[1]: 1e16,
                            electricity_activities[2]: 1e16,
                            electricity_activities[3]: 1e16}}

In [9]:
# Instantiate and solve the problem (here with HiGHS)
instance = pulpo_worker.instantiate(choices=choices, demand=demand)
results = pulpo_worker.solve()

Creating Instance
Instance created


In [10]:
# Summarize the results
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)

The following demand / functional unit has been specified: 


,Demand
"market for electricity, high voltage | electricity, high voltage | DE",1.288190e+11



These are the impacts contained in the objective:


,Key,Value
0,"('IPCC 2013', 'climate change', 'GWP 100a')",1.599836e+10



The following impacts were calculated: 


,Key,Value
4,"('ReCiPe Midpoint (E) V1.13', 'ionising radiat...",5.414918e+10
0,"('IPCC 2013', 'climate change', 'GWP 100a')",1.599836e+10
1,"('ReCiPe Endpoint (E,A)', 'human health', 'tot...",2.410592e+09
3,"('ReCiPe Endpoint (E,A)', 'resources', 'total')",6.763921e+08
2,"('ReCiPe Endpoint (E,A)', 'ecosystem quality',...",6.219297e+08



The following choices were made: 
electricity


electricity                \
                                                     Activity      Capacity   
Activity 0  electricity production, nuclear, pressure wate...  1.000000e+16   

                          
                   Value  
Activity 0  8.350729e+10

No additional constraints have been passed.


In [11]:
pulpo_worker.save_results(choices=choices, demand=demand, name='energy_transition_showcase.xlsx')

### No optimization

In [12]:
# Instantiate and solve the problem (here with HiGHS)
instance = pulpo_worker.instantiate(choices={}, demand=demand)
results = pulpo_worker.solve()

Creating Instance
Instance created


In [13]:
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)

The following demand / functional unit has been specified: 


,Demand
"market for electricity, high voltage | electricity, high voltage | DE",1.288190e+11



These are the impacts contained in the objective:


,Key,Value
0,"('IPCC 2013', 'climate change', 'GWP 100a')",7.254725e+10



The following impacts were calculated: 


,Key,Value
0,"('IPCC 2013', 'climate change', 'GWP 100a')",7.254725e+10
1,"('ReCiPe Endpoint (E,A)', 'human health', 'tot...",2.066171e+10
4,"('ReCiPe Midpoint (E) V1.13', 'ionising radiat...",1.371322e+10
2,"('ReCiPe Endpoint (E,A)', 'ecosystem quality',...",2.945184e+09
3,"('ReCiPe Endpoint (E,A)', 'resources', 'total')",2.439436e+09



The following choices were made: 
electricity


electricity                \
                                                     Activity      Capacity   
Activity 0  electricity production, lignite | electricity,...  1.000000e+16   
Activity 1  electricity production, nuclear, pressure wate...  1.000000e+16   
Activity 2  electricity production, hard coal | electricit...  1.000000e+16   
Activity 3  electricity production, wind, 1-3MW turbine, o...  1.000000e+16   

                          
                   Value  
Activity 0  3.257648e+10  
Activity 1  1.424185e+10  
Activity 2  1.605628e+10  
Activity 3  2.073959e+10

No additional constraints have been passed.


In [14]:
pulpo_worker.save_results(choices=choices, demand=demand, name='energy_transition_showcase_noopt.xlsx')